# fb testing

In [5]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import pickle
import dask as pd
import pandas as pd_old
import warnings
import utils
import heartpy as hp
from ECG_feature_extractor_1001 import *
# import time 
import time
from datetime import datetime
from biosppy.signals import ecg
from feature_extraction import SubjectData, compute_features, get_samples, combine_files

# To ignore all warnings:
warnings.filterwarnings("ignore", module="numpy")

In [6]:
WINDOW_IN_SECONDS = 60
stride = 1
label_dict = {'baseline': 1, 'stress': 2, 'amusement': 0}
int_to_label = {1: 'baseline', 2: 'stress', 0: 'amusement'}
feat_names = None
loadPath = '../data/WESAD'
savePath = '../data/GN-WESAD'
subject_feature_path = '/subject_feats'
onedrive = '/mnt/d/Users/alkurdi/OneDrive - University of Illinois - Urbana/data/GN-WESAD'
n_samples = 10 
subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
snrs = [ 0.0001, 0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6] #0.00001,
fb_model_list = ['DT', 'RF', 'SVM', 'LDA', 'KNN', 'AdaBoost']

if not os.path.exists(savePath):
    os.makedirs(savePath)
if not os.path.exists(savePath + subject_feature_path):
    os.makedirs(savePath + subject_feature_path)

In [3]:
def get_processing_status(snrs, subject_ids, onedrive, n_samples= 10):
    #bads are the ones that do not have the gaussian-modified data.  
    bads = []
    bad_snrs = []
    bad_subjects = []
    bad_ns = []
    completed_snrs = []
    for n_i in range(n_samples):
        for snr in snrs:
            for subject_id in subject_ids:
                #print(snr)
                
                #print(f'{onedrive}/n_{n_i}/snr_{snr}/S{subject_id}/{a}')
                try: 
                    a = os.listdir(f'{onedrive}/n_{n_i}/snr_{str(snr)}/S{subject_id}')
                    a[0]
                    completed_snrs.append(snr)
                except:
                    bads.append(f'n_{n_i}/snr_{snr}/S{subject_id}')
                    bad_snrs.append(snr)
                    bad_subjects.append(subject_id)
                    bad_ns.append(n_i)

    bad_snrs = sorted(set(bad_snrs))
    bad_subjects = sorted(set(bad_subjects))
    bad_ns = sorted(set(bad_ns))
    completed_snrs = sorted(set(completed_snrs))
    #printing after checking
    print(f'completed snrs :{completed_snrs}')
    print(f'incomplete snrs :{bad_snrs}')
get_processing_status(snrs, subject_ids, onedrive, n_samples= n_samples)

completed snrs :[0.0001, 0.001, 0.01, 0.05, 0.1, 0.15, 0.2, 0.3, 0.4, 0.5, 0.6]
incomplete snrs :[]


In [4]:
dataset_list = []
for i in os.listdir(f'{loadPath}/subject_feats'):
    #print (i)
    if 'S' not in i[0]:
        dataset_list.append(i)
        
            
            

In [5]:
model_results = {}

for each_dataset in dataset_list:
    for snr in snrs:
        for model in fb_model_list:
            pass 

In [6]:
results_table = pd_old.DataFrame(columns=['SNR', 'Accuracy', 'F1-Score', 'dataset'])
results_table.loc[str('SVM')] = pd_old.Series({'SNR':1, 'Accuracy':5, 'F1 Score':2, 'dataset':'WESAD'})
results_table.loc[str('RF')] = pd_old.Series({'SNR':1, 'Accuracy':5, 'F1 Score':2, 'dataset':'WESAD'})
'''
fb_model_list = ['SVM', 'RF']
for model in fb_model_list:
    for i in range(len(snrs)):
        results_table.loc[str(model) + str(snrs[i])] = pd.Series({'SNR':snrs[i], 'Accuracy':svm_accuracy[i], 'F1 Score':2, 'dataset':'WESAD'})
'''
display(results_table)

,SNR,Accuracy,F1-Score,dataset
SVM,1,5,NaN,WESAD
RF,1,5,NaN,WESAD


In [7]:

start = time.time()

ecg_biosppy = ecg
ecg = None
fs_ecg = 700
fs_ppg = 64
subject_id=2 


In [8]:
subject = SubjectData(main_path=loadPath, subject_number=subject_id)
data_dict = subject.get_wrist_and_chest_data()

In [ ]:
ecg = data_dict['ECG'][40000:60000].flatten()
ppg = data_dict['BVP'][int(40000/700*64):int(60000/700*64)].flatten()
print('ecg len', len(ecg))
print('ppg len', len(ppg))


#testing for ECG

now = time.time()
#wd4, m4 = hp.process(ecg, fs_ecg)
#print('hp.process execution time is %5.2fs' % (time.time()-now))

now = time.time()
#pack2, ecgout2, time_dict2 = freq_ratio_hybrid(ecg, fs_ecg, RR1, method='welch', factor = 1)
#print('freq_ratio_hybrid execution time is %5.2fs' % (time.time()-now))



In [ ]:
now = time.time()
BS_signal_analysis3, pack3, ecg_out3 = analyze_ecg(ecg, fs_ecg)
print('analyze_ecg execution time is ', now-time.time())


In [ ]:
ecg = data_dict['ECG'][10000:80000].flatten()


In [140]:
#testing for PPG

now = datetime.now()
pack, ppg, RR, time_dict = freq_ratio( ppg, fs_ppg, method='welch', factor = 1)
print('freq_ratio execution time is ', now-time.time())

now = datetime.now()
pack, ppg, time_dict = freq_ratio_hybrid(ppg, fs_ppg, method='welch', factor = 1)
print('freq_ratio_hybrid execution time is ', now-time.time())

now = datetime.now()
BS_signal_analysis, pack, ppg_out = analyze_ecg(ppg, fs_ppg)
print('analyze_ecg execution time is ', now-time.time())

now = datetime.now()
wd, m = hp.process(ppg, fs_ppg)
print('hp.process execution time is ', now-time.time())

now = datetime.now()
pack, ppg, RR = freq_ratio_fast( ppg, fs_ppg, method='welch', factor = 1)
print('freq_ratio_fast execution time is ', now-time.time())

SyntaxError: invalid syntax (2154369309.py, line 4)

In [7]:
subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17]
import logging


def combine_noiZ_files(subject_ids):
    today = datetime.now().strftime('%Y-%m-%d')
    logging.basicConfig(level=logging.DEBUG, filename=today+'-combine.log', filemode='w', force=True)
    logging.info('Started')
    print('total number of combines: ', len(snrs)*n_samples)
    df_list = []
    i = 0
    for snr in snrs:
        for n_i in range(n_samples):
            for s in subject_ids:
                df = pd_old.read_csv(f'{savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/S{s}_feats.csv', index_col=0)
                df['subject'] = s
                df_list.append(df)
            df = pd_old.concat(df_list)
            df['label'] = (df['0'].astype(str) + df['1'].astype(str) + df['2'].astype(str)).apply(lambda x: x.index('1'))
            df.drop(['0', '1', '2'], axis=1, inplace=True)
            df.reset_index(drop=True, inplace=True)
            now = datetime.now().strftime('%Y-%m-%d')
            df.to_csv(f'{savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/{now}_feats_filt.csv')
            i+=1
            logging.info(f'Saved file to: {savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/{today}_feats_filt.csv  {i}/{len(snrs)*n_samples}')
            print('Saved file to: ',f'.../n_{n_i}/snr_{snr}{subject_feature_path}/{now}_feats_filt.csv   {i}/{len(snrs)*n_samples}')
            counts = df['label'].value_counts()
            logging.info('Number of samples per class:')
            logging.info('baseline: {0[1]}; stress: {1[1]}; amusement: {2[1]} '.format(*list(zip(counts.index, counts.values))))
    logging.info('all done!')

In [8]:
combine_noiZ_files(subject_ids)
# took 19m 30s to run

total number of combines:  110
Saved file to:  .../n_0/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_1/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_2/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_3/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_4/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_5/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_6/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_7/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_8/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_9/snr_0.0001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_0/snr_0.001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_1/snr_0.001/subject_feats/2023-11-12_feats_filt.csv
Saved file to:  .../n_2/snr_0.001/subject_feats/2023-11-12_feats_filt.c

In [233]:
import pandas as pd
import os
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor
import logging
now = datetime.now().strftime('%Y-%m-%d')

def process_subject_file(snr, n_i, s):
    file_path = f'{savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/S{s}_feats.csv'
    if os.path.exists(file_path):
        df = pd.read_csv(file_path, index_col=0)
        df['subject'] = s
        return df
    return None

def combine_noiZ_files(subjects):
    now = datetime.now().strftime('%Y-%m-%d')
    logging.basicConfig(level=logging.INFO, filename=now+'-combine.log', filemode='w', force=True)

    for snr in snrs:
        for n_i in range(n_samples):
            # Parallelize file reading
            with ThreadPoolExecutor(max_workers=10) as executor:
                futures = [executor.submit(process_subject_file, snr, n_i, s) for s in subjects]
                df_list = [future.result() for future in futures if future.result() is not None]

            if df_list:
                df = pd.concat(df_list)
                df['label'] = df[['0', '1', '2']].idxmax(axis=1)
                df.drop(['0', '1', '2'], axis=1, inplace=True)
                df.reset_index(drop=True, inplace=True)
                
                df.to_csv(f'{savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/{now}_feats_filt.csv')
                logging.info('-' * 20)
                logging.info(f'Saved file to: {savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/{now}_feats_filt.csv')
                
                counts = df['label'].value_counts()
                logging.info('Number of samples per class:')
                logging.info('baseline: {0[1]}; stress: {1[1]}; amusement: {2[1]} '.format(*list(zip(counts.index, counts.values))))
    logging.info('all done!')

In [223]:
n_i = 5
s = 2
snr = 0.6
df = pd_old.read_csv(f'{savePath}/n_{n_i}/snr_{snr}{subject_feature_path}/S{subject_id}_feats.csv', index_col=0)
df['subject'] = s
#df_list.append(df)
#f = pd_old.concat(df_list)
df['label'] = (df['0'].astype(str) + df['1'].astype(str) + df['2'].astype(str)).apply(lambda x: x.index('1'))
df.drop(['0', '1', '2'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)

counts = df['label'].value_counts()
print('Number of samples per class:')
print('baseline: {0[1]}; stress: {1[1]}; amusement: {2[1]} '.format(*list(zip(counts.index, counts.values))))

Number of samples per class:
baseline: 119; stress: 61; amusement: 34 


In [234]:
combine_noiZ_files(subject_ids)
#this paralellized took 34seconds

In [31]:
i = 0

def smash(s):
    global i 
    s = s+1
    i+=1
    #s = damnit(s)
    print(i)
    return s
def damnit(s):
    global i 
    s = s+10
    i=s
    print(i)
    return s

for f in range(10):
    smash(2)

print(i)


1
2
3
4
5
6
7
8
9
10
10


In [33]:
logging.basicConfig(level=logging.INFO, filemode='w', force=True)
logging.info('GN-WESAD models ran and results generated and saved in: ')

INFO:root:GN-WESAD models ran and results generated and saved in: 


In [49]:
now = datetime.now().strftime('%Y.%m.%d-%H.%M%p')
now

'2023.11.13-05.22AM'